# Dati Anac + Banca dati Servizio Contratti Pubblici - SCP
## Data processing

Il risultante di questo notebook è un dataset che processa i dati provenienti dalla Banca dati Servizio Contratti Pubblici relativi ad un certo indice PA congiunti con i dati estratti dal db di ANAC.


### Banca dati Servizio Contratti Pubblici - SCP

Il Dataset contiene :

- gli avvisi
- i bandi
- gli esiti di gara 

in formato aperto raccolti dalla Banca dati SCP - Servizio Contratti Pubblici gestita dalla Direzione Generale per la regolazione e i contratti pubblici del Ministero delle Infrastrutture e Trasporti. 

Il Servizio Contratti Pubblici (SCP) consente la pubblicazione on line di bandi di gara, avvisi sui risultati delle procedure di affidamento e programmi triennali di opere pubbliche. 

Maggiori informazioni sono disponibili sul sito www.serviziocontrattipubblici.it che consente alle stazioni appaltanti di pubblicare gli avvisi ed i bandi per lavori, servizi e forniture ed altre informazioni. 

Servizio Contratti Pubblici è una delle banche dati nazionali a titolarità del Ministero delle Infrastrutture e Trasporti elencate nell'allegato B del D.Lgs n.97/2016 (art. 9) che modifica l’obbligo di pubblicazione previsto nella normativa sulla trasparenza D.Lgs n.33/2013 (art. 9bis). I dati sono aggiornati quotidianamente.
http://dati.mit.gov.it/catalog/dataset/scp

https://www.serviziocontrattipubblici.it

https://contrattipubblici.org

## General

In [77]:
#IPython setting
import pandas as pd
import numpy as np
import json 
import os
import re
import seaborn as sns
import matplotlib as mpl 
import matplotlib.cm as cm 
import matplotlib.pyplot as plt 
import matplotlib.style as style
import seaborn as sns 
%matplotlib inline 

from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #setta l'esecuzione di più statement in unica cella
pd.set_option('display.max_colwidth', -1) 
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60) 

# Set up formatting so larger numbers aren't displayed in scientific notation (h/t @thecapacity)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

sns.set_context("notebook")
sns.set()
sns.set_style("darkgrid")
sns.set_palette(sns.color_palette("Set2", 10))
style.use('fivethirtyeight')

### Filtro su una determinata struttura 

### http://www.indicepa.gov.it

[INDICE PA Comune di Torino](http://www.indicepa.gov.it/elenco-cod-fisc/n-elenco-cf.php?cod_amm=c_l219)

CITTA' DI TORINO - APPALTI LAVORI PUBBLICI - indice PA: 00514490010

In [78]:
filtroCfStrutturaProponente="00514490010"

## Dati ANAC

In [79]:
anac = pd.read_csv("data/anacDataset_postprocessed.tsv", sep='\t', low_memory=False)

In [80]:
list(anac)

['cig',
 'cfStrutturaProponente',
 'denominazioneStrutturaProponente',
 'oggetto',
 'sceltaContraente',
 'importoAggiudicazione',
 'importoSommeLiquidate',
 'dataInizio',
 'dataUltimazione',
 'jsonPartecipanti',
 'jsonAggiudicatari',
 'totalePartecipanti',
 'totaleAggiudicatari',
 'totaleRaggruppamentopartecipanti',
 'totaleRaggruppamentoAggiudicatari',
 'cfPrimoaggiudicatario',
 'denominazionePrimoaggiudicatario']

In [81]:
anac.oggetto.value_counts()

Spese economia 2017                                                                                                                                                                                                                                     3689
Spese effettuate in economia anno 2015                                                                                                                                                                                                                  2999
OGGETTO NON PRESENTE                                                                                                                                                                                                                                    2880
Spese economia 2016                                                                                                                                                                                                                              

In [82]:
anac.denominazionePrimoaggiudicatario.value_counts()

Raggruppamento                                                   270812
MAGGIOLI SPA                                                     14373 
TELECOM ITALIA SPA                                               9881  
RADIOTAXI 3570  SOCIETA COOPERATI                                7953  
POSTE ITALIANE SPA                                               6983  
SIGMA ALDRICH SRL                                                6507  
MYO SRL                                                          3764  
GRUPPO SPAGGIARI PARMA SPA                                       3755  
SOLUZIONE UFFICIO SRL                                            3714  
ERREBIAN SPA                                                     3665  
EUROCLONE SPA                                                    3490  
WALDNER TECNOLOGIE MEDICALI SRL                                  3342  
JOHNSON e JOHNSON MEDICAL SPA                                    3293  
BORGIONE CENTRO DIDATTICO SRL                                   

In [83]:
anac.denominazioneStrutturaProponente.value_counts()

AZIENDA SANITARIA DELLA PROVINCIA AUTONOMA DI BOLZANO                                                                                                                          52370
ACEA ATO 2 SPA                                                                                                                                                                 45975
RFI Rete Ferroviaria Italiana                                                                                                                                                  31447
ACEA DISTRIBUZIONE SPA                                                                                                                                                         28170
AUSL della Romagna                                                                                                                                                             25005
Universit degli Studi di Torino                                                                

In [84]:
anac.jsonPartecipanti

0          {"raggruppamento":[],"partecipante":[{"codiceFiscale":"01615440342","identificativoFiscaleEstero":null,"ragioneSociale":"Callegari S.r.l."},{"codiceFiscale":"01331270999","identificativoFiscaleEstero":null,"ragioneSociale":"MINIAS GLOBE DIAGNOSTICS SRL"},{"codiceFiscale":"00747170157","identificativoFiscaleEstero":null,"ragioneSociale":"ROCHE DIAGNOSTICS SPA"}]}
1          {"raggruppamento":[],"partecipante":[{"codiceFiscale":"06450560963","identificativoFiscaleEstero":null,"ragioneSociale":"CV office SAS di Del Freo Chiara E C."}]}                                                                                                                                                                                                          
2          {"raggruppamento":[],"partecipante":[{"codiceFiscale":"08539010010","identificativoFiscaleEstero":null,"ragioneSociale":"Vodafone Italia S.p.A."},{"codiceFiscale":"08141031214","identificativoFiscaleEstero":null,"ragioneSociale":"Fastweb

### Filtro su struttura proponente

In [85]:
#seziono su stazione appaltante di torino
anac_to = anac[anac.cfStrutturaProponente==filtroCfStrutturaProponente]

In [86]:
anac_to.head(3)

,cig,cfStrutturaProponente,denominazioneStrutturaProponente,oggetto,sceltaContraente,importoAggiudicazione,importoSommeLiquidate,dataInizio,dataUltimazione,jsonPartecipanti,jsonAggiudicatari,totalePartecipanti,totaleAggiudicatari,totaleRaggruppamentopartecipanti,totaleRaggruppamentoAggiudicatari,cfPrimoaggiudicatario,denominazionePrimoaggiudicatario
2723943,48346363D6,00514490010,Citt di Torino,M S DIFFUSA IMPIANTI ANTINCENDIO DELLE SCUOLE CITTADINE PER VERIFICHE DI LEGGE ADEGUAMENTI TECNICI E RIPARAZIONI ANNO 2012,01PROCEDURA APERTA,364146.260,215704.570,20/03/2015,18/03/2016,"{""raggruppamento"":[],""partecipante"":[{""codiceFiscale"":""02993580014"",""identificativoFiscaleEstero"":null,""ragioneSociale"":""CUDIA IMPIANTI DI CUDIA FRANCESCO""}]}","{""aggiudicatarioRaggruppamento"":[],""aggiudicatario"":[{""codiceFiscale"":""02993580014"",""identificativoFiscaleEstero"":null,""ragioneSociale"":""CUDIA IMPIANTI DI CUDIA FRANCESCO""}]}",1,1,0,0,02993580014,CUDIA IMPIANTI DI CUDIA FRANCESCO
2723944,ZF407586BD,00514490010,Citt di Torino,SISTEMAZIONE VIE NUOVA COMUNALIZZ LOTTO 7 B BIL 2009 VIA SANTAGATA SPOSTAMENTO UTENZA A SEGUITO ESPROPRIO AFFIDAMENTO AD ENI SPA IMPORTO DI,04PROCEDURA NEGOZIATA SENZA PREVIA PUBBLICAZIONE DEL BANDO,27451.270,0.000,19/03/2013,29/09/2014,"{""raggruppamento"":[],""partecipante"":[{""codiceFiscale"":""00905811006"",""identificativoFiscaleEstero"":null,""ragioneSociale"":""ENI S.P.A.""}]}","{""aggiudicatarioRaggruppamento"":[],""aggiudicatario"":[{""codiceFiscale"":""00905811006"",""identificativoFiscaleEstero"":null,""ragioneSociale"":""ENI S.P.A.""}]}",1,1,0,0,00905811006,ENI SPA
2723945,465112341E,00514490010,Citt di Torino,SERVIZIO DI CONSULTAZIONE BANCHE DATI,04PROCEDURA NEGOZIATA SENZA PREVIA PUBBLICAZIONE DEL BANDO,80727.400,80727.400,31/12/2012,31/12/2013,"{""raggruppamento"":[],""partecipante"":[{""codiceFiscale"":""01995120019"",""identificativoFiscaleEstero"":null,""ragioneSociale"":""C.S.I. PIEMONTE CONSORZIO PER IL SISTEMA INFORMATIVO""}]}","{""aggiudicatarioRaggruppamento"":[],""aggiudicatario"":[{""codiceFiscale"":""01995120019"",""identificativoFiscaleEstero"":null,""ragioneSociale"":""C.S.I. PIEMONTE CONSORZIO PER IL SISTEMA INFORMATIVO""}]}",1,1,0,0,01995120019,CSI PIEMONTE CONSORZIO PER IL SISTEMA INFORMATIVO


# Dati SCP

In particolare voglio analizzare il dataset esiti contenente gli esiti delle gare.

In [189]:
avvisi = pd.read_csv("data/ServizioContrattiPubblici/scpavvisi.csv")
bandi = pd.read_csv("data/ServizioContrattiPubblici/scpbandi.csv")
esiti = pd.read_csv("data/ServizioContrattiPubblici/scpesiti.csv")

/Users/mariaclaudiabodino/anaconda3/envs/mappa/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [191]:
##ex: '"2009-11-19T23:00:00.000Z"'
d_format = '%Y-%m-%dT%H:%M:%S.%f'

In [192]:
avvisi_str_labels = ["codice_fiscale_stazione_appaltante","data_pubb_esito_scp", "data_pubb_guri", "data_pubb_albo", 'data_agg_def',"termine_pres_dom_off", 'data_pres_fine']
bandi_str_labels = ["codice_fiscale_stazione_appaltante","data_pubb_bando_scp","data_pubb_guue", "data_pubb_guri", "data_pubb_albo", "termine_pres_dom_off"]
esiti_str_labels = ["codice_fiscale_stazione_appaltante","data_pubb_esito_scp","data_pubb_esito_guue","data_pubb_guri","data_pubb_albo","data_agg_def", "data_pres_fine", "termine_pres_dom_off"]

In [193]:
avvisi_data_labels = ["data_pubb_esito_scp","data_pubb_esito_guue", "data_pubb_guri", "data_pubb_albo", 'data_agg_def',"termine_pres_dom_off", 'data_pres_fine']
bandi_data_labels = ["data_pubb_bando_scp","data_pubb_guue", "data_pubb_guri", "data_pubb_albo", "termine_pres_dom_off"]
esiti_data_labels = ["data_pubb_esito_scp","data_pubb_esito_guue","data_pubb_guri","data_pubb_albo","data_agg_def", "data_pres_fine", "termine_pres_dom_off"]

### Preprocessing fix malformatted fields and Data

In [195]:
#clean double quotes
for l in avvisi_str_labels:
    avvisi[l]=avvisi[l].str.replace("\"","")
for l in bandi_str_labels:
    bandi[l]=bandi[l].str.replace("\"","")
for l in esiti_str_labels:
    esiti[l]=esiti[l].str.replace("\"","")
    

In [196]:
d_format = '%Y-%m-%dT%H:%M:%S.%f'
#If ‘coerce’, then invalid parsing will be set as NaN
for l in avvisi_data_labels:
    avvisi[l]= pd.to_datetime(avvisi[l], format=d_format, errors='coerce')
    avvisi[l]= avvisi[l].dt.date
#If ‘coerce’, then invalid parsing will be set as NaN
for l in bandi_data_labels:
    bandi[l]= pd.to_datetime(bandi[l], format=d_format, errors='coerce')
    bandi[l]= bandi[l].dt.date
#If ‘coerce’, then invalid parsing will be set as NaN
for l in esiti_data_labels:
    esiti[l]= pd.to_datetime(esiti[l], format=d_format, errors='coerce')    
    esiti[l]= esiti[l].dt.date

In [93]:
len(avvisi)
len(bandi)
len(esiti)

123972

183381

123834

In [94]:
avvisi.to_csv("data/ServizioContrattiPubblici/processed/avvisi.csv",index=False)

In [95]:
esiti.to_csv("data/ServizioContrattiPubblici/processed/esiti.csv",index=False)

In [96]:
bandi.to_csv("data/ServizioContrattiPubblici/processed/bandi.csv",index=False)

### Cleaning: quick checks "esiti"


In [197]:
esiti.dtypes
esiti.shape
np.sum([esiti.imp_lavori.isnull()])
np.sum([esiti.imp_contrattuale.isnull()])

id_esito                              int64  
codice_fiscale_stazione_appaltante    object 
denominazione_stazione_appaltante     object 
nr_lotto                              int64  
tipo_bando                            object 
tipo_settore                          object 
oggetto                               object 
cpv                                   object 
categoria                             object 
rup                                   object 
cup                                   object 
cig                                   object 
data_pubb_esito_scp                   object 
luogo_esecuzione                      object 
imp_lavori                            float64
imp_contrattuale                      float64
ribasso_agg                           float64
tipo_procedura                        object 
asta_elettronica                      object 
forcella                              object 
appalto_riservato                     object 
data_pubb_esito_guue              

(123834, 38)

2118

1714

### Filtro dataset su parametro "filtroCfStrutturaProponente" impostato 

In [198]:
avvisi_to = avvisi[avvisi.codice_fiscale_stazione_appaltante==filtroCfStrutturaProponente]
bandi_to = bandi[bandi.codice_fiscale_stazione_appaltante==filtroCfStrutturaProponente]
esiti_to = esiti[esiti.codice_fiscale_stazione_appaltante==filtroCfStrutturaProponente]

In [199]:
len(avvisi_to)
len(bandi_to)
len(esiti_to)
np.sum([esiti_to.imp_lavori.isnull()])
np.sum([esiti_to.imp_contrattuale.isnull()])

901

1184

901

0

4

In [200]:
bandi_esiti_to = pd.merge(bandi_to, esiti_to, on='cig', how='left')
bandi_esiti_to.shape

(134701, 78)

## JOIN ANAC e ESITI

In questo caso voglio estratte un csv finale dato da tutti gli esiti di gara presenti per la PA scelta e gli eventuali dati presenti per i CIG elencati provenienti dal dataset di ANAC

In [201]:
bandi_esiti_to = pd.merge(esiti_to, bandi_to, on='cig', how='left')
bandi_esiti_to.shape

(134635, 78)

In [202]:
#bandi_anac_TO = pd.merge(bandi_to,anac_to, on='cig', how='left')

In [203]:
esiti_anac_TO = pd.merge(esiti_to,anac_to, on='cig', how='left')
len(esiti_anac_TO)
len(esiti_to)

1092

901

In [204]:
esiti_anac_TO.dtypes

id_esito                              int64  
codice_fiscale_stazione_appaltante    object 
denominazione_stazione_appaltante     object 
nr_lotto                              int64  
tipo_bando                            object 
tipo_settore                          object 
oggetto_x                             object 
cpv                                   object 
categoria                             object 
rup                                   object 
cup                                   object 
cig                                   object 
data_pubb_esito_scp                   object 
luogo_esecuzione                      object 
imp_lavori                            float64
imp_contrattuale                      float64
ribasso_agg                           float64
tipo_procedura                        object 
asta_elettronica                      object 
forcella                              object 
appalto_riservato                     object 
data_pubb_esito_guue              

In [205]:
esiti_anac_TO.denominazione_stazione_appaltante.value_counts()

CITTA' DI TORINO - FORNITURE E SERVIZI        625
CITTA' DI TORINO - APPALTI LAVORI PUBBLICI    467
Name: denominazione_stazione_appaltante, dtype: int64

In [206]:
esiti_anac_TO.luogo_esecuzione.value_counts()

TORINO                906
CREVOLADOSSOLA        11 
SAN MAURO TORINESE    1  
BIBIANA               1  
Name: luogo_esecuzione, dtype: int64

In [207]:
esiti_anac_TO.tipo_evento.value_counts()

Gara aggiudicata    1087
Gara deserta        5   
Name: tipo_evento, dtype: int64

In [208]:
esiti_anac_TO.tipo_bando.value_counts()

Servizi      555
Lavori       466
Forniture    71 
Name: tipo_bando, dtype: int64

In [209]:
esiti_anac_TO.tipo_procedura.value_counts()

Procedura aperta                                                   903
Procedura ristretta                                                148
Procedura negoziata previa pubblicazione                           25 
Confronto competitivo in adesione ad accordo quadro/convenzione    15 
Dialogo competitivo D.Lgs 163/2006 art.58                          1  
Name: tipo_procedura, dtype: int64

In [210]:
esiti_anac_TO.categoria.value_counts()

Fornitura di servizi                                                                                    557
OG3 - Strade, autostrade, ponti, viadotti, ferrovie, metropolitane, funicolari, piste aeroportuali e    166
OG1 - Edifici civili e industriali                                                                      100
Fornitura di beni                                                                                       69 
OG2 - Restauro e manutenzione dei beni immobili sottoposti a tutela ai sensi delle disposizioni in m    20 
OS24 - Verde e arredo urbano                                                                            8  
OS6 - Finiture di opere generali in materiali lignei, plastici, metallici e vetrosi                     4  
OS23 - Demolizione di opere                                                                             2  
OS3 - Impianti idrico - sanitario, cucine, lavanderie                                                   1  
OS2-A - Superfici decorate d

In [211]:
esiti_anac_TO.criterio_aggiudicazione.value_counts()

Prezzo più basso                          660
Offerta economicamente più vantaggiosa    431
Name: criterio_aggiudicazione, dtype: int64

In [212]:
esiti_anac_TO.tipo_settore.value_counts()

Ordinario    1091
Speciale     1   
Name: tipo_settore, dtype: int64

In [213]:
esiti_anac_TO.sceltaContraente.value_counts()

01PROCEDURA APERTA                                                191
02PROCEDURA RISTRETTA                                             67 
04PROCEDURA NEGOZIATA SENZA PREVIA PUBBLICAZIONE DEL BANDO        42 
26AFFIDAMENTO DIRETTO IN ADESIONE AD ACCORDO QUADROCONVENZIONE    3  
23AFFIDAMENTO IN ECONOMIA  AFFIDAMENTO DIRETTO                    2  
Name: sceltaContraente, dtype: int64

In [214]:
esiti_anac_TO.to_csv("SCP_Esiti_Anac_"+filtroCfStrutturaProponente+".csv", index=False, sep=";")


In [215]:
esiti_anac_TO.head(20)

,id_esito,codice_fiscale_stazione_appaltante,denominazione_stazione_appaltante,nr_lotto,tipo_bando,tipo_settore,oggetto_x,cpv,categoria,rup,cup,cig,data_pubb_esito_scp,luogo_esecuzione,imp_lavori,imp_contrattuale,ribasso_agg,tipo_procedura,asta_elettronica,forcella,appalto_riservato,data_pubb_esito_guue,data_pubb_guri,data_pubb_albo,url_profilo_comm,data_agg_def,termine_pres_dom_off,n_impr_rich,n_impr_inv,n_impr_off,n_impr_amm,criterio_aggiudicazione,data_pres_fine,note_agg,tipo_evento,motivazione,aggiudicatario,url,cfStrutturaProponente,denominazioneStrutturaProponente,oggetto_y,sceltaContraente,importoAggiudicazione,importoSommeLiquidate,dataInizio,dataUltimazione,jsonPartecipanti,jsonAggiudicatari,totalePartecipanti,totaleAggiudicatari,totaleRaggruppamentopartecipanti,totaleRaggruppamentoAggiudicatari,cfPrimoaggiudicatario,denominazionePrimoaggiudicatario
0,17650,00514490010,CITTA' DI TORINO - APPALTI LAVORI PUBBLICI,0,Lavori,Ordinario,PROCEDURA APERTA N. 53/2009 - COMPLESSO SCOLASTICA DI CORSO BRAMANTE 75 - MANUTENZIONE STRAORDINARIA RECUPERO FUNZIONALE E ADEGUAMENTO NORMATIVO,45453100-8,OG2 - Restauro e manutenzione dei beni immobili sottoposti a tutela ai sensi delle disposizioni in m,PONCINI PIERLUIGI,C35J04000310004,NaN,2009-07-23,NaN,1734957.100,1299787.450,26.374,Procedura aperta,No,No,No,NaN,2009-05-12,2009-05-12,www.comune.torino.it/appalti,2009-07-14,2009-06-08,0.000,0.000,4.000,4.000,Prezzo più basso,2011-03-14,NaN,Gara aggiudicata,NaN,A.T.I. GRUPPO TECNOIMPRESE SRL,https://www.serviziocontrattipubblici.it/PubbAvvisiBandiEsiti/GetScheda.do?codice=17650&entita=ESITI,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,NaN
1,17902,00514490010,CITTA' DI TORINO - APPALTI LAVORI PUBBLICI,1,Lavori,Ordinario,"PROCEDURA APERTA N.57/2009 - INTERVENTI STRAORDINARI SULLE PAVIMENTAZIONI DELLE VIE, STRADE E PIAZZE DELLA CITTA' - 12 LOTTI - BILANCIO 2008",45233140-2,"OG3 - Strade, autostrade, ponti, viadotti, ferrovie, metropolitane, funicolari, piste aeroportuali e",BERTASIO ROBERTO,NaN,NaN,2009-08-02,NaN,6850000.000,270581.000,42.265,Procedura aperta,No,No,No,NaN,2009-05-03,2009-05-03,http://www.comune.torino.it/appalti,2009-07-21,2009-06-15,0.000,0.000,16.000,16.000,Prezzo più basso,2010-05-30,LOTTO N. 1 - CIRCOSCRIZIONI 1,Gara aggiudicata,NaN,ANTARES SRL,https://www.serviziocontrattipubblici.it/PubbAvvisiBandiEsiti/GetScheda.do?codice=17902&entita=ESITI,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,NaN
2,17915,00514490010,CITTA' DI TORINO - APPALTI LAVORI PUBBLICI,1,Lavori,Ordinario,PROCEDURA APERTA N. 63/2009- INTERVENTI DI ORDINARIA MANUTENZIONE DEGLI EDIFICI SCOLASTICI DI OGNI GENERE SITI NELLE CIRCORSCRIZIONI 1-10 E SOGGIORNI LIGURI.,45400000-1,OG1 - Edifici civili e industriali,PONCINI PIERLUIGI,C15D09000000004,NaN,2009-08-02,NaN,623525.800,46802.630,31.024,Procedura aperta,No,No,No,NaN,2009-06-09,2009-06-09,www.comune.torino.it/appalti,2009-07-28,2009-07-06,0.000,0.000,6.000,6.000,Prezzo più basso,2010-07-30,LOTTO N. 1,Gara aggiudicata,NaN,MAGNETTI SRL,https://www.serviziocontrattipubblici.it/PubbAvvisiBandiEsiti/GetScheda.do?codice=17915&entita=ESITI,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,NaN
3,17916,00514490010,CITTA' DI TORINO - APPALTI LAVORI PUBBLICI,2,Lavori,Ordinario,PROCEDURA APERTA N. 63/2009- INTERVENTI DI ORDINARIA MANUTENZIONE DEGLI EDIFICI SCOLASTICI DI OGNI GENERE SITI NELLE CIRCORSCRIZIONI 1-10 E SOGGIORNI LIGURI.,45400000-1,OG1 - Edifici civili e industriali,PONCINI PIERLUIGI,C15D09000000004,NaN,2009-08-02,NaN,623525.800,947320016.000,40.911,Procedura aperta,No,No,No,NaN,2009-06-09,2009-06-09,www.comune.torino.it/appalti,2009-07-28,2009-07-06,0.000,0.000,8.000,8.000,Prezzo più basso,2010-07-30,LOTTO N. 2,Gara aggiudicata,NaN,CONSORZIO CARDEA S.C.,https://www.serviziocontrattipubblici.it/PubbAvvisiBandiEsiti/GetScheda.do?codice=17916&entita=ESITI,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,NaN
4,17917,00514490010,CITTA' DI TORINO - APPALTI LAVOR

## EDA

In [ ]:
ax = sns.countplot(x=esiti_anac_TO.data_pubb_albo.dt.year, data=esiti_anac_TO)
ax.set_title("Numero di esiti per anno (anno pubblicazione albo)");
ax.set_ylabel("Numero di esiti");
ax.set_xlabel("Anno");

In [ ]:
#bandi per tipologia di intervento
ax = sns.countplot(x=esiti_anac_TO.data_pubb_albo.dt.year, data=esiti_anac_TO,  hue=esiti_anac_TO.categoria)
ax.set_title("Esiti per anno e tipologia di intervento");
ax.set_ylabel("Numero di esiti");
ax.set_xlabel("Anno");

In [ ]:
#band per fascia di importo
#bandi per anno: numero e importo
#Numeri Bandi di gara per tipologia: 
#Importi Bandi di gara per tipologia:

### OpenCUP

I record ai quali è stato assegnato un cup sono molto pochi, non ha quindi rilevanza nello stato attuale il join con il dataset OPENCUP

In [ ]:
esiti_to.cup.isnull().sum()
bandi_to.cup.isnull().sum()
len(bandi_to)

In [ ]:
cup = pd.read_csv("data/cup/AREA_NORD_EST_CSV.csv", sep='|', header=0, error_bad_lines=False)
cup.shape()